In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import r2_score
from ml_metrics import rmse
from tqdm.notebook import tqdm
from ipywidgets import interact
import pickle

%matplotlib inline

time -> timestamp, sekundy  
pm25_nie -> pm25 z konkursu

In [2]:
'''
wszystkie:
test jest bezpośrednio po train
wielkości podajemy w godzinach
w ostatniej paczce jest pusty test
przekazuje do modelu cały dataframe
'''


def customCV(df, h_train, h_test):                  # 4.260244256299462 0.9387329666074647 05:37 pm25 basicFE log=False 30*24
                                                    # 4.445078039582411 0.933301413124673 05:32 pm25 basicFE log=True
                                                    # 5.4802462497424225 0.8986186663982577 05:31 pm25 no_futFE log=False
    '''
    tworzy paczki train o określonej wielkosci
    train musi być większy od test
    kolejny train zaczyna się od poprzedniego test
    miedzy test są przerwy
    train nie nakładają się na siebie
    '''
    
    assert h_test < h_train, "test > train"
    parts_nr = (df.shape[0] // h_train) + 1
    for i in np.arange(0, parts_nr):         
        train_begin = i * h_train
        train_end = train_begin + h_train
        test_end = train_end + h_test       
        yield df[train_begin : train_end], df[train_end : test_end]
        

def customCV2(df, h_train, h_test):                 # 5.024122777097885 0.9052867780165903 2:39:17 pm25 no_futFE log=False
                                                    # 5.092291999780713 0.9026991306509206 2:43:10 pm25 basicFE log=False
    '''
    tworzy paczki train o określonej wielkosci
    train musi być większy od test
    kolejny train kończy się poprzednim test
    kolejne test leżą jeden obok drugiego
    train mogą się nakładać na siebie jeżeli h_train > h_test (czyli zwykle)
    '''
    assert h_test < h_train, "test > train"
    parts_nr = df.shape[0] // h_test
    for i in np.arange(0, parts_nr):         
        train_begin = i * h_test 
        train_end = train_begin + h_train
        test_end = train_end + h_test
        yield df[train_begin : train_end], df[train_end : test_end]
        if train_end > df.shape[0]: break           
            

def customCV3(df, h_train, h_test):                  # 6.310146602536797 0.8655883998947564 22:58 pm25 basicFE log=False
    '''
    tworzy paczki train o rosnącej długości
    miedzy test są przerwy
    '''
    parts_nr = (df.shape[0] // h_train) + 1
    for i in np.arange(0, parts_nr):         
        train_begin = i * h_train
        train_end = train_begin + h_train
        test_end = train_end + h_test       
        yield df[0 : train_end], df[train_end : test_end]       
        

def customCV4(df, h_train, h_test):
    '''
    tworzy paczki train o rosnącej długości
    kolejne test leżą jeden obok drugiego
    '''
    parts_nr = df.shape[0] // h_test
    for i in np.arange(0, parts_nr):         
        train_begin = i * h_test 
        train_end = train_begin + h_train
        test_end = train_end + h_test
        yield df[0 : train_end], df[train_end : test_end]
        if train_end > df.shape[0]: break

In [3]:
def basicFE():
    
    df = pd.read_csv('data_pm25_plus_darksky.csv', index_col = 0)
    df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
    
    df.drop( columns=['time', 'year', 'month', 'day', 'hour', 'day_of_week', 'no_of_week'], inplace = True)
       
#     fragment kodu dodający prognozę pogody
#
#     weather_forecast = pd.read_csv('warsaw_full.csv', index_col = 0)
#     weather_forecast['date_time'] = pd.to_datetime(weather_forecast['date_time'], format="%Y-%m-%d %H:%M:%S")   
#     df = pd.merge(df, weather_forecast, left_on='date', right_on='date_time')
#     df.drop( columns=['date_time', 'moonrise', 'moonset', 'sunrise', 'sunset'], inplace = True)    
#     forecast_feats = ['maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex_y', 'uvIndex.1', 'moon_illumination', 
#                        'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC', 'WindGustKmph', 'cloudcover', 'humidity_y', 
#                        'precipMM', 'pressure_y', 'tempC', 'visibility_y', 'winddirDegree', 'windspeedKmph']
    
    # powtórzone kolumny na ewentualność korzystania z innego pliku żródłowego
    df['hour']    = df.date.dt.hour
    df['day_m']   = df.date.dt.day
    df['month']   = df.date.dt.month
    df['year']    = df.date.dt.year
    df['week_y']  = df.date.dt.week
    df['day_y']   = df.date.dt.dayofyear
    df['day_w']   = df.date.dt.dayofweek
    df['quarter'] = df.date.dt.quarter
    df['hour_w']  = df.date.dt.hour + df.date.dt.dayofweek * 24
    df['weekend'] = df.date.dt.dayofweek >= 5
    
    obj_feats = df.select_dtypes(np.object).columns
    for feat in obj_feats:
        df[feat + '_cat'] = df[feat].factorize()[0]
        
    df['pm25'] = (df['pm25_nie'] + df['pm25_kon'] + df['pm25_wok']) / 3
    
    for feat in ['pm25_nie', 'pm25_kon', 'pm25_wok', 'pm25']:
        df[feat + '_log'] = np.log(df[ df[feat]>0 ][feat]+1)
    
    future_feats = ['quarter', 'month', 'week_y', 'day_y', 'day_m', 'day_w', 'hour', 'hour_w', 'year', 'weekend'] # + forecast_feats

    black_list_feats = ['pm25_nie', 'pm25_kon', 'pm25_wok', 'pm25_nie_log', 'pm25_kon_log', 'pm25_wok_log']       # 'date'?
    
    num_feats = df.select_dtypes(include=[np.number, bool]).columns.values
    num_feats = [feat for feat in num_feats if feat not in black_list_feats]

    return df, future_feats, num_feats, black_list_feats

In [4]:
def sec_prize_FE(avg_len=6, roll_nr=[6, 12], agr_fn=['min', 'max', 'mean', 'median']):
    
    df, future_feats, num_feats, black_list_feats = basicFE()
    
    df['windSpeed_log'] = (np.log(df[ df['windSpeed']>0 ]['windSpeed']+5))
    df['windGust_log']  = (np.log(df[ df['windGust']>0  ]['windGust']+2))
           
    past_feats = df.select_dtypes(include=[np.number, bool]).columns.values
    past_feats = [feat for feat in past_feats if feat not in black_list_feats]
    past_feats = [feat for feat in past_feats if feat not in future_feats]   
    roll_columns = [feat for feat in past_feats if not feat.endswith('_cat')]   
    
    for r_feat in roll_columns:
        for fn in agr_fn:
            for nr in roll_nr:    
                suffix = '_roll{}{}'.format(nr, fn)
                df[r_feat + suffix] = getattr(df[r_feat].rolling(nr), fn)()
    roll_feats = [feat for feat in df.columns if '_roll' in feat]
        
    df['avg_mov'] = df['pm25'].rolling( window=avg_len, min_periods=1).mean()
    df['avg_mov'] = df.apply(lambda x: np.nan if np.isnan(x['pm25']) else x['avg_mov'], axis=1)
    df['avg_diff'] = df.apply(lambda x: x['pm25']-x['avg_mov'], axis=1)
    
    num_feats = df.select_dtypes(include=[np.number, bool]).columns.values
    num_feats = [feat for feat in num_feats if feat not in black_list_feats]
    
    return df, future_feats, num_feats, black_list_feats

In [5]:
# przesuwa dane które znamy w przyszłości (zmienne zależne od czasu, prognozy, ewentualnie wyliczone średnie np. dla dni tygodnia)

def shift_X(X, feats, nr=-1):
    for feat in feats:
        X[feat] = X[feat].shift(nr)
    return X

In [6]:
def model_fit(forecast_len, train_factor, df, model, log, validation, num_feats, future_feats, y_feat):
    print(df.shape, len(future_feats), len(num_feats), forecast_len, train_factor, log, y_feat, validation)
    models = []
    for i in np.arange(forecast_len):
            models.append(model)
    
    y_test_all, y_pred_all = np.array([]), np.array([])
    test_len = forecast_len + 1    
    for train, test in tqdm(list(validation(df, train_factor*forecast_len, test_len))):
        if test.shape[0] == 0: continue        # pomija ostatnią paczkę z pustym test
        '''
        Łączymy train i test. 
        Przylegają one do siebie.
        Musimy mieć dane z przyszłości aby wypełniały luki dla future_feats po .shift(-1)
        '''        
        X = pd.concat([ train[num_feats], test[num_feats] ])        # dataframe, .values robimy dalej
        y = (pd.concat([ train[y_feat], test[y_feat] ]).values).astype(np.float64)
        y_test_all = np.concatenate((y_test_all, y[-forecast_len :]))
        if log:
            y = (pd.concat([ train[y_feat + '_log'], test[y_feat + '_log'] ]).values).astype(np.float64)
            
        y_pred = np.array([])
        for i in np.arange(forecast_len):                     # każdy model szkolony oddzielnie
            X = shift_X(X, future_feats, -1)                        # przesuwa kolumny znane w przyszłości - future_feats
            y = pd.Series(y).shift(-1).values                       # przesuwa y_feat o 1h w przeszłość
            X_pred  = X[ -test_len : -test_len+1 ].values           # pierwszy wiersz z bloku test - do predykcji
            X_train = X[ 0 : -test_len ].values                     # pomija zbiór testowy            
            y_train = y[ 0 : -test_len ]                       
            models[i].fit(X_train, y_train)                        
            y_hour = (models[i].predict(X_pred)).astype(np.float64) # prognoza na 1h
            y_pred = np.concatenate((y_pred, y_hour))                              

        if log:
            y_pred = np.exp(y_pred)-1
        y_pred_all = np.concatenate((y_pred_all, y_pred))
        
    r2_all = r2_score(y_test_all, y_pred_all)
    rmse_all = rmse(y_test_all, y_pred_all)
    print(rmse_all, r2_all)
    return models, rmse_all, r2_all, y_test_all, y_pred_all

In [7]:
df, future_feats, num_feats, black_list_feats = basicFE()

In [8]:
# testowanie kolejnych funkcji dla CV

fn = customCV                                # customCV, customCV2, customCV3, customCV4

for i, (train, test) in enumerate(fn(df, 3000, 1000), 1):
    print (i, train.index[0], train.index[-1], train.shape, test.shape)

1 0 2999 (3000, 41) (1000, 41)
2 3000 5999 (3000, 41) (1000, 41)
3 6000 8999 (3000, 41) (1000, 41)
4 9000 11999 (3000, 41) (1000, 41)
5 12000 14999 (3000, 41) (1000, 41)
6 15000 17999 (3000, 41) (1000, 41)
7 18000 20999 (3000, 41) (1000, 41)
8 21000 23999 (3000, 41) (1000, 41)
9 24000 26999 (3000, 41) (1000, 41)
10 27000 29999 (3000, 41) (1000, 41)
11 30000 32999 (3000, 41) (1000, 41)
12 33000 35999 (3000, 41) (1000, 41)
13 36000 38999 (3000, 41) (1000, 41)
14 39000 41999 (3000, 41) (1000, 41)
15 42000 43852 (1853, 41) (0, 41)


### Co będziemy prognozować?

In [9]:
df[['pm25_nie', 'pm25_kon', 'pm25_wok', 'pm25']].describe()

,pm25_nie,pm25_kon,pm25_wok,pm25
count,43853.000000,43853.000000,43853.000000,43853.000000
mean,25.296143,31.188994,19.677586,25.387574
std,16.907685,26.346212,14.572871,16.320308
min,0.877018,0.010000,1.281766,1.649104
25%,13.535946,11.300000,9.438459,12.487519
50%,20.661466,20.960000,15.640000,21.800467
75%,31.902000,46.402488,25.220000,34.412381
max,187.930147,256.028970,155.365434,155.512509


In [10]:
model_params = {'learning_rate': 0.2,
                  'max_depth': 5,
                  'n_estimators': 100,
                  'n_jobs': 8,
                  'objective': 'reg:squarederror',
                  'random_state': 2020,
                  'subsample': 0.8,
                 }

model = xgb.XGBRegressor(**model_params)

In [11]:
# tylko kolumny pochodne od daty, bez danych z przyszłości

name = 'timeOnly_CV_24_30_TF_noFF_pm25_False'
time_feats = ['quarter', 'month', 'week_y', 'day_y', 'day_m', 'day_w', 'hour', 'hour_w', 'year', 'weekend']
val = customCV
df, future_feats, num_feats, black_list_feats = basicFE()
models, rmse_all, r2_all, y_test_all, y_pred_all = model_fit(forecast_len=24, train_factor=30, df=df, model=model, log=False, 
                                                             validation=val, num_feats=time_feats, future_feats=[], 
                                                             y_feat='pm25')
# result = models, rmse_all, r2_all, y_test_all, y_pred_all
# with open(name, 'wb') as f:
#     pickle.dump(result, f)

(43853, 41) 0 10 24 30 False pm25 <function customCV at 0x00000246C52A6438>



6.086012115997185 0.8749673379068365


In [79]:
# tylko kolumny pochodne od daty, uwzględnia je jako dane z przyszłości

name = 'timeOnly_CV_24_30_TF_TF_pm25_False'
time_feats = ['quarter', 'month', 'week_y', 'day_y', 'day_m', 'day_w', 'hour', 'hour_w', 'year', 'weekend']
val = customCV
df, future_feats, num_feats, black_list_feats = basicFE()
models, rmse_all, r2_all, y_test_all, y_pred_all = model_fit(forecast_len=24, train_factor=30, df=df, model=model, log=False, 
                                                             validation=val, num_feats=time_feats, future_feats=time_feats, 
                                                             y_feat='pm25')
result = models, rmse_all, r2_all, y_test_all, y_pred_all
with open(name, 'wb') as f:
    pickle.dump(result, f)

(43853, 41) 10 10 24 30 False pm25 <function customCV at 0x0000024B3B82BC18>



3.9915168753407064 0.9462183908589014


In [83]:
# tylko kolumny pochodne od daty, uwzględnia je jako dane z przyszłości, użycie logarytmu pm25

name = 'timeOnly_CV_24_30_TF_TF_pm25_True'
time_feats = ['quarter', 'month', 'week_y', 'day_y', 'day_m', 'day_w', 'hour', 'hour_w', 'year', 'weekend']
val = customCV
df, future_feats, num_feats, black_list_feats = basicFE()
models, rmse_all, r2_all, y_test_all, y_pred_all = model_fit(forecast_len=24, train_factor=30, df=df, model=model, log=True, 
                                                             validation=val, num_feats=time_feats, future_feats=time_feats, 
                                                             y_feat='pm25')
result = models, rmse_all, r2_all, y_test_all, y_pred_all
with open(name, 'wb') as f:
    pickle.dump(result, f)

(43853, 41) 10 10 24 30 True pm25 <function customCV at 0x0000024B3B82BC18>



3.907159401497947 0.9484676305232014


In [80]:
# tylko kolumny pochodne od pogody, bez danych pochodnych od daty, bez danych z przyszłości

name = 'weatherOnly_CV_24_30_WF_noFF_pm25_False'
val = customCV
df, future_feats, num_feats, black_list_feats = basicFE()
time_feats = ['quarter', 'month', 'week_y', 'day_y', 'day_m', 'day_w', 'hour', 'hour_w', 'year', 'weekend']
past_feats = [feat for feat in num_feats if feat not in time_feats]
models, rmse_all, r2_all, y_test_all, y_pred_all = model_fit(forecast_len=24, train_factor=30, df=df, model=model, log=False, 
                                                             validation=val, num_feats=past_feats, future_feats=[], 
                                                             y_feat='pm25')
result = models, rmse_all, r2_all, y_test_all, y_pred_all
with open(name, 'wb') as f:
    pickle.dump(result, f)

(43853, 41) 0 21 24 30 False pm25 <function customCV at 0x0000024B3B82BC18>



5.763302230744239 0.8878754682603945


In [81]:
# kolumny pochodne od pogody i czasu, bez danych z przyszłości

name = 'Basic_CV_24_30_NF_noFF_pm25_False'
val = customCV
df, future_feats, num_feats, black_list_feats = basicFE()
models, rmse_all, r2_all, y_test_all, y_pred_all = model_fit(forecast_len=24, train_factor=30, df=df, model=model, log=False, 
                                                             validation=val, num_feats=num_feats, future_feats=[], 
                                                             y_feat='pm25')
result = models, rmse_all, r2_all, y_test_all, y_pred_all
with open(name, 'wb') as f:
    pickle.dump(result, f)

(43853, 41) 0 31 24 30 False pm25 <function customCV at 0x0000024B3B82BC18>



5.4802462497424225 0.8986186663982577


In [82]:
# kolumny pochodne od pogody i czasu, kolumny pochodne od daty uwzględnia jako dane z przyszłości

name = 'Basic_CV_24_30_NF_FF_pm25_False'
val = customCV
df, future_feats, num_feats, black_list_feats = basicFE()
models, rmse_all, r2_all, y_test_all, y_pred_all = model_fit(forecast_len=24, train_factor=30, df=df, model=model, log=False, 
                                                             validation=val, num_feats=num_feats, future_feats=future_feats, 
                                                             y_feat='pm25')
result = models, rmse_all, r2_all, y_test_all, y_pred_all
with open(name, 'wb') as f:
    pickle.dump(result, f)

(43853, 41) 10 31 24 30 False pm25 <function customCV at 0x0000024B3B82BC18>



4.260244256299462 0.9387329666074647


In [89]:
# FE z mojego rozwiazania konkursowego, kolumny pochodne od daty uwzględnia jako dane z przyszłości

name = 'sec_prize_FE_24_30_NF_FF_pm25_False'
val = customCV
df, future_feats, num_feats, black_list_feats = sec_prize_FE(avg_len=6, roll_nr=[24], agr_fn=['median'])
models, rmse_all, r2_all, y_test_all, y_pred_all = model_fit(forecast_len=24, train_factor=30, df=df, model=model, log=False, 
                                                             validation=val, num_feats=num_feats, future_feats=future_feats, 
                                                             y_feat='pm25')
result = models, rmse_all, r2_all, y_test_all, y_pred_all
with open(name, 'wb') as f:
    pickle.dump(result, f)

(43853, 65) 10 55 24 30 False pm25 <function customCV at 0x0000024B3B82BC18>



4.163986964861951 0.941470262641485


In [90]:
# FE, model i y_feat z mojego rozwiazania konkursowego, kolumny pochodne od daty uwzględnia jako dane z przyszłości
# optymalizacja parametrów robiona dla zbioru danych z lukami

model_params_sec = {'learning_rate': 0.2476843816313341, 
                     'max_depth': 5, 
                     'n_estimators': 100, 
                     'n_jobs': 8, 
                     'objective': 'reg:squarederror', 
                     'random_state': 70, 
                     'subsample': 0.8561623057645249
                 }
model_sec = xgb.XGBRegressor(**model_params_sec)

name = 'sec_prizeAll_CV_24_30_NF_FF_pm25_nie_False'
val = customCV
df, future_feats, num_feats, black_list_feats = sec_prize_FE(avg_len=6, roll_nr=[24], agr_fn=['median'])
models, rmse_all, r2_all, y_test_all, y_pred_all = model_fit(forecast_len=24, train_factor=30, df=df, model=model_sec, log=False, 
                                                             validation=val, num_feats=num_feats, future_feats=future_feats, 
                                                             y_feat='pm25_nie')
result = models, rmse_all, r2_all, y_test_all, y_pred_all
with open(name, 'wb') as f:
    pickle.dump(result, f)

(43853, 65) 10 55 24 30 False pm25_nie <function customCV at 0x0000024B3B82BC18>



5.607968530042018 0.8924652718716108


In [12]:
name = 'timeOnly_CV_24_30_TF_TF_pm25_True'                                # best CV 3.907159401497947 0.9484676305232014
with open(name, 'rb') as f:
    models, rmse_all, r2_all, y_test_all, y_pred_all = pickle.load(f)
    
name = 'sec_prize_FE_24_30_NF_FF_pm25_False'                              # best weather solution 4.163986964861951 0.941470262641485
with open(name, 'rb') as f:
    models2, rmse_all2, r2_all2, y_test_all2, y_pred_all2 = pickle.load(f)

In [13]:
list([(test, pred) for (test, pred) in zip(y_test_all, y_pred_all)])

[(30.042918333333333, 26.492968279701298),
 (29.010210333333333, 25.421618511442517),
 (30.508806000000003, 28.02408247455936),
 (31.451367333333334, 25.822052032758023),
 (32.113413333333334, 26.44566999760123),
 (29.301482333333336, 26.1581122777098),
 (26.576297999999998, 27.476613890923627),
 (21.79640733333333, 26.977427105945836),
 (19.474676333333335, 23.51849190962964),
 (24.255493, 22.22753194612502),
 (26.167371000000003, 24.760264768764447),
 (29.322088666666662, 26.618852784840975),
 (29.129427000000003, 26.526575775566723),
 (24.80447666666667, 28.541613155293472),
 (23.246936333333334, 29.501791088816894),
 (25.060288333333336, 26.486132437359828),
 (25.872633000000004, 30.42385955857315),
 (22.229646666666667, 28.573035793571123),
 (23.849515666666665, 23.679528690122652),
 (25.103612666666667, 24.96128990633227),
 (24.81377333333333, 26.136073854454498),
 (25.621723333333332, 24.57248751056132),
 (28.57757433333333, 25.149503008952035),
 (29.893376333333332, 35.97348818

In [14]:
def insert_every_day(y, part_size=24, insert_size=24):
    result = np.array([])
    insert = np.array( [np.nan]*insert_size )
    for one in np.arange(0, len(y-part_size), part_size):
        result = np.append( result, y[one:one+part_size])
        result = np.append( result, insert)
    return result

In [15]:
@interact()

def all_comparison(real=True, real2=True, pred=True, pred2=True):

    y_test_all_ = insert_every_day(y_test_all, 24, 1)
    y_test_all2_ = insert_every_day(y_test_all2, 24, 1)
    y_pred_all_ = insert_every_day(y_pred_all, 24, 1)
    y_pred_all2_ = insert_every_day(y_pred_all2, 24, 1)
    
    plt.figure(figsize=(20,10))
    if real: plt.plot(np.arange(len(y_test_all_)), y_test_all_, label='real', c='blue');
    if real2: plt.plot(np.arange(len(y_test_all2_)), y_test_all2_, label='real2', c='blue');
    if pred: plt.plot(np.arange(len(y_pred_all_)), y_pred_all_, label='pred', c='red');
    if pred2: plt.plot(np.arange(len(y_pred_all2_)), y_pred_all2_, label='pred2', c='lime');
    plt.legend()
    plt.show
    print('pred: {:.3f} pred2: {:.3f}'.format(rmse(y_test_all, y_pred_all), rmse(y_test_all2, y_pred_all2)))

interactive(children=(Checkbox(value=True, description='real'), Checkbox(value=True, description='real2'), Che…

In [16]:
@interact(part_nr=(0, 59))

def partial_comparison(part_nr=0, real=True, real2=True, pred=True, pred2=True):
    long = 24       
    idx = part_nr*long
    y_real  =  y_test_all[ idx : idx+long ]
    y_real2 = y_test_all2[ idx : idx+long ]
    y_pred  =  y_pred_all[ idx : idx+long ]
    y_pred2 = y_pred_all2[ idx : idx+long ]
    
    plt.figure(figsize=(20,10))
    if real: plt.plot(np.arange(long), y_real, label='real', c='blue');
    if real2: plt.plot(np.arange(long), y_real2, label='real2', c='blue');
    if pred: plt.plot(np.arange(long), y_pred, label='pred', c='red');
    if pred2: plt.plot(np.arange(long), y_pred2, label='pred2', c='lime');
    
    plt.legend();
    print('nr: {} pred: {:.3f} pred2: {:.3f}'.format(part_nr, rmse(y_real, y_pred), rmse(y_real2, y_pred2)))

interactive(children=(IntSlider(value=0, description='part_nr', max=59), Checkbox(value=True, description='rea…

In [ ]:
nr